In [5]:
from datasets import load_dataset

cc_news = load_dataset("cc_news")

Dataset cc_news downloaded and prepared to /Users/mismayil/.cache/huggingface/datasets/cc_news/plain_text/1.0.0/ae469e556251e6e7e20a789f93803c7de19d0c4311b6854ab072fecb4e401bd6. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00,  4.06it/s]


In [15]:
from torch.utils.data import DataLoader

loader = DataLoader(cc_news["train"], shuffle=True, batch_size=100)

In [16]:
samples = next(iter(loader))

len(samples["description"])

100

In [17]:
with open("data/cc_news_samples.txt", "w") as f:
    f.write("\n".join([sample for sample in samples["description"] if sample.strip()]))

In [18]:
daily_dialog = load_dataset("daily_dialog", split="test")

Dataset daily_dialog downloaded and prepared to /Users/mismayil/.cache/huggingface/datasets/daily_dialog/default/1.0.0/1d0a58c7f2a4dab5ed9d01dbde8e55e0058e589ab81fce5c2df929ea810eabcd. Subsequent calls will reuse this data.


In [19]:
daily_dialog

Dataset({
    features: ['dialog', 'act', 'emotion'],
    num_rows: 1000
})

In [22]:
loader = DataLoader(daily_dialog, shuffle=True, batch_size=50)

In [29]:
daily_dialog[:2]

{'dialog': [['Hey man , you wanna buy some weed ? ',
   ' Some what ? ',
   ' Weed ! You know ? Pot , Ganja , Mary Jane some chronic ! ',
   ' Oh , umm , no thanks . ',
   ' I also have blow if you prefer to do a few lines . ',
   ' No , I am ok , really . ',
   ' Come on man ! I even got dope and acid ! Try some ! ',
   ' Do you really have all of these drugs ? Where do you get them from ? ',
   ' I got my connections ! Just tell me what you want and I ’ ll even give you one ounce for free . ',
   ' Sounds good ! Let ’ s see , I want . ',
   ' Yeah ? ',
   ' I want you to put your hands behind your head ! You are under arrest ! '],
  ['The taxi drivers are on strike again . ',
   ' What for ? ',
   ' They want the government to reduce the price of the gasoline . ',
   ' It is really a hot potato . ']],
 'act': [[3, 2, 3, 4, 3, 4, 3, 2, 3, 4, 2, 3], [1, 2, 1, 1]],
 'emotion': [[0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0], [0, 0, 0, 0]]}

In [30]:
dialog_samples = ["".join(dialog) for dialog in daily_dialog[:50]["dialog"]]

In [31]:
len(dialog_samples)

50

In [32]:
with open("data/dialog_samples.txt", "w") as f:
    f.write("\n".join([sample for sample in dialog_samples if sample.strip()]))

In [33]:
roc_stories = load_dataset("wza/roc_stories")

No config specified, defaulting to: roc_stories/all


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 320.00it/s]


Dataset roc_stories downloaded and prepared to /Users/mismayil/.cache/huggingface/datasets/wza___roc_stories/all/2.1.0/43e2851d9f31e08e4b2dd07a8057ed7a64cbb25cc7105d09856c14e638695506. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 113.57it/s]


In [34]:
roc_stories

DatasetDict({
    train: Dataset({
        features: ['storyid', 'storytitle', 'sentence1', 'sentence2', 'sentence3', 'sentence4', 'sentence5'],
        num_rows: 98161
    })
})

In [37]:
roc_samples = roc_stories["train"][:50]
roc_samples = [" ".join(sample) for sample in zip(roc_samples["sentence1"], roc_samples["sentence2"], roc_samples["sentence3"], roc_samples["sentence4"], roc_samples["sentence5"])]

In [38]:
roc_samples[:3]

["Dan's parents were overweight. Dan was overweight as well. The doctors told his parents it was unhealthy. His parents understood and decided to make a change. They got themselves and Dan on a diet.",
 "Carrie had just learned how to ride a bike. She didn't have a bike of her own. Carrie would sneak rides on her sister's bike. She got nervous on a hill and crashed into a wall. The bike frame bent and Carrie got a deep gash on her leg.",
 "Morgan enjoyed long walks on the beach. She and her boyfriend decided to go for a long walk. After walking for over a mile, something happened. Morgan decided to propose to her boyfriend. Her boyfriend was upset he didn't propose to her first."]

In [ ]:
with open("data/story_samples.txt", "w") as f:
    f.write("\n".join([sample for sample in roc_samples if sample.strip()]))

In [3]:
import spacy

text = "PersonX falls in love with PersonY"
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

for token in doc:
    print(token.text, token.dep_, [child for child in token.children])

PersonX nsubj []
falls ROOT [PersonX, in, with]
in prep [love]
love pobj []
with prep [PersonY]
PersonY pobj []


In [2]:
from kogito.core.processors.head import NounPhraseHeadExtractor, VerbPhraseHeadExtractor
import spacy

nlp = spacy.load("en_core_web_sm")

ext = NounPhraseHeadExtractor("ext", nlp)
vext = VerbPhraseHeadExtractor("vext", nlp)
vext.extract("PersonX falls in love with PersonY")

[to fall]

In [4]:
from kogito.inference import CommonsenseInference

# Initialize inference module with a spacy language pipeline
csi = CommonsenseInference(language="en_core_web_sm")

# Run inference
text = "PersonX becomes a great basketball player"
kgraph = csi.infer(text, match_relations=False)

# Save output knowledge graph to JSON file
kgraph.to_jsonl("results/test.json")

Extracting knowledge heads...


In [10]:
from kogito.inference import CommonsenseInference
from kogito.core.knowledge import KnowledgeGraph
from kogito.models.gpt3.zeroshot import GPT3Zeroshot
from kogito.core.relation import KnowledgeRelation, register_relation, EVENT_RELATIONS
import time, os

# csi = CommonsenseInference()
# csi.remove_processor("simple_relation_matcher")

def x_wishes_verbalizer(head, **kwargs):
   # index will be passed from the model
   # so that we can enumerate samples which helps with inference
   index = kwargs.get("index")
   index_txt = f"{index}" if index is not None else ""
   return f"Situation {index_txt}: {head}\nWishes: As a result, PersonX wishes"

X_WISHES = KnowledgeRelation("xWishes",
                             verbalizer=x_wishes_verbalizer,
                             prompt="How does this situation affect each character's wishes?")
register_relation(X_WISHES, kind="social")
print(X_WISHES in EVENT_RELATIONS)

True
